<a href="https://colab.research.google.com/github/shooby-d/projects/blob/main/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detecting Poaching 😺

# Preparing drive🚦

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


General imports

In [ ]:
import pandas as pd 
import os
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
from pathlib import Path



#Label extracting script...creates new directory 'yolo_labels' where everything will be stored💎

In [ ]:
import json

# path to annotations file
train_annFile = Path('/content/drive/MyDrive/annotations/instances_train.json')
val_annFile = Path('/content/drive/MyDrive/annotations/instances_val.json')
test_annFile = Path('/content/drive/MyDrive/annotations/instances_test.json')

# path to directory containing images
train_imgDir = Path('/content/drive/MyDrive/images/train')
val_imgDir = Path('/content/drive/MyDrive/images/val')
test_imgDir = Path('/content/drive/MyDrive/images/test')

# output directory for YOLO-formatted labels
outDir = Path('/content/drive/MyDrive/yolo_labels')

if not os.path.exists(outDir): 
    os.makedirs(outDir)

# load COCO annotations
for annFile, imgDir, label_suffix in [(train_annFile, train_imgDir, '_train'), (val_annFile, val_imgDir, '_val'), (test_annFile, test_imgDir, '_test')]:
    with open(annFile, 'r') as f:
        coco = json.load(f)

    outDir_labeled = outDir / ('labeled' + label_suffix)
    print(outDir_labeled) # print new directories
    if not os.path.exists(outDir_labeled):
        os.makedirs(outDir_labeled)

    # loop over images
    for img in coco['images']:
        img_id = img['id']
        img_file = Path(img['file_name'])
        img_path = imgDir / img_file

        # create YOLO label file
        out_file = outDir_labeled / (img_file.stem + '.txt')

        # create label string
        label_str = ''
        found_annotation = False
        for ann in coco['annotations']:
            if ann['image_id'] == img_id:
                bbox = ann['bbox']
                x_center = bbox[0] + bbox[2] / 2
                y_center = bbox[1] + bbox[3] / 2
                w = bbox[2]
                h = bbox[3]

                # convert to YOLO format
                x_center /= img['width']
                y_center /= img['height']
                w /= img['width']
                h /= img['height']
                label_str += f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n"
                found_annotation = True

        # write label string to file
        if found_annotation:
            with open(out_file, 'w') as f:
                f.write(label_str)
        else:
            print(f"No annotations found for image {img_file}")

print("Labeling complete!") # takes a while for 13,111 train, 2,755 val, and 2,764 test images


/content/drive/MyDrive/yolo_labels/labeled_train
/content/drive/MyDrive/yolo_labels/labeled_val
/content/drive/MyDrive/yolo_labels/labeled_test
Labeling complete!


Checking directory

In [ ]:
%cd /content/drive/MyDrive/yolo_labels
%ls 

/content/drive/MyDrive/yolo_labels
labeled_test/  labeled_train/  labeled_val/


#Adding the model (YOLOv8n) 😎

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ultralytics import YOLO

#Code for directory moving (organizing data)🚘

In [ ]:
import shutil


Merging images with annotations

In [ ]:
source_dir = '/content/drive/MyDrive/images/train' # shared from fruitpunch
dest_dir = '/content/drive/MyDrive/yolo_labels/labeled_train'

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

png_files = [f for f in os.listdir(source_dir) if f.endswith('.PNG')]
if png_files:
    for f in png_files:
        shutil.copy(os.path.join(source_dir, f), dest_dir)
        print(f"Copied {f} to {dest_dir}")
else:
    print("No .PNG files found in source directory.")


Streaming output truncated to the last 5000 lines.
Copied videobh_1_004905.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_1_003149.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_3_003500.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_42_000207.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_41_000253.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_1_000191.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_3_003772.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_41_000089.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied video53_1_001193.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied videobh_2_000145.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied video53_2_001996.PNG to /content/drive/MyDrive/yolo_labels/labeled_train
Copied video53_1_002573.PNG to /content/drive/MyDrive/yolo_labels/

In [ ]:
source_dir = '/content/drive/MyDrive/images/val'
dest_dir = '/content/drive/MyDrive/yolo_labels/labeled_val'

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

png_files = [f for f in os.listdir(source_dir) if f.endswith('.PNG')]
if png_files:
    for f in png_files:
        shutil.copy(os.path.join(source_dir, f), dest_dir)
        print(f"Copied {f} to {dest_dir}")
else:
    print("No .PNG files found in source directory.")


Copied videobh_41_000262.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_1_003236.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_3_000245.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_2_000273.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_4_000236.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_1_003251.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_3_003472.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_2_000441.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_1_001107.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_3_000127.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_1_000131.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied videobh_1_001715.PNG to /content/drive/MyDrive/yolo_labels/labeled_val
Copied video53_5_000060.PNG to /content/drive/MyDrive/yolo_labe

In [ ]:
source_dir = '/content/drive/MyDrive/images/test'
dest_dir = '/content/drive/MyDrive/yolo_labels/labeled_test'
# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

png_files = [f for f in os.listdir(source_dir) if f.endswith('.PNG')]
if png_files:
    for f in png_files:
        shutil.copy(os.path.join(source_dir, f), dest_dir)
        print(f"Copied {f} to {dest_dir}")
else:
    print("No .PNG files found in source directory.")

Copied video82_2_000431.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_3_000881.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_2_000636.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_2_000610.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_2_000101.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_3_000054.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_2_000678.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_2_000123.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_3_000757.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_3_000511.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_1_000668.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_3_000527.PNG to /content/drive/MyDrive/yolo_labels/labeled_test
Copied video82_1_000723.PNG to /content/drive/MyDriv

In [ ]:
%cd /content/drive/MyDrive/yolo_labels/labeled_train
%ls

In [ ]:
%cd /content/drive/MyDrive/yolo_labels/labeled_val
%ls

In [ ]:
%cd /content/drive/MyDrive/yolo_labels/labeled_test
%ls

#YOLO style file creation👽

In [ ]:
import yaml # some items will be in different spots in final product

data = dict(
    # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
    path='/content/drive/MyDrive/FruitPunchAI',
    train='yolo_labels/labeled_train',
    val='yolo_labels/labeled_val',
    test='yolo_labels/labeled_test',

    
    # Classes
    names=['Human'],

)

# write to file
with open('/content/drive/MyDrive/yolodata.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


end of pre-processing data

#Training the model🧠

In [ ]:
model = YOLO('yolov8n.pt', task="detect")
results = model.train(data='/content/drive/MyDrive/yolo.yaml', epochs=25, imgsz=640, workers=2, save=True) # had to move locations in drive that's why the data files dont have the same name
results = model.val()

100%|██████████| 6.23M/6.23M [00:00<00:00, 13.0MB/s]
Ultralytics YOLOv8.0.83 🚀 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/yolo.yaml, epochs=25, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, forma

In [ ]:
results = model("/content/drive/MyDrive/yolo_labels/labeled_test/video82_3_000517.PNG")


image 1/1 /content/drive/MyDrive/yolo_labels/labeled_test/video82_3_000517.PNG: 480x640 6 Humans, 181.8ms
Speed: 0.6ms preprocess, 181.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
success=model.export(format="onnx")

Ultralytics YOLOv8.0.83 🚀 Python-3.9.16 torch-2.0.0+cu118 CPU

PyTorch: starting from runs/detect/train/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: export success ✅ 0.8s, saved as runs/detect/train/weights/best.onnx (11.7 MB)

Export complete (1.0s)
Results saved to /content/drive/MyDrive/yolo_labels/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=/content/drive/MyDrive/yolo.yaml 
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



#Best weights stored in yolo_labels directory under runs/detect/train/weights 🍉

In [ ]:
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

ANSI_X3.4-1968


In [ ]:
model = YOLO('/content/drive/MyDrive/FruitPunchAI/poacherdetector.onnx', task='detect')  # load a custom model

# Testing the model 🐸

In [ ]:
results = model("/content/drive/MyDrive/FruitPunchAI/yolo_labels/labeled_test") # test model

Loading /content/drive/MyDrive/FruitPunchAI/poacherdetector.onnx for ONNX Runtime inference...


    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1900 /content/drive/MyDrive/FruitPunchAI/yolo_labels/labeled_test/video82_1_000470.PNG: 640x640 2 Humans, 765.8ms
image 2/1900 /content/drive/MyDrive/FruitPunchAI/yolo_labels/labeled_test/video82_1_000471.PNG: 640x640 2 Humans, 201.3ms
image 3/1900 /content/drive/MyDrive/FruitPunchAI/yolo_labels/labeled_test/video82_1_000472.PNG: 640x640 4 Humans, 190.4m

In [ ]:
import cv2
res_plotted = results[0].plot()